In [1]:
# Import libraries
import os
import librosa
import csv

In [2]:
# Read file names
path = os.path
files = {}

data_dir = os.listdir(
    path.join(os.getcwd(), '..', 'data')
)

for dir in data_dir:
    audio_files = os.listdir(
        path.join(os.getcwd(), '..', 'data', dir)
    )

    files[dir] = []
    files[dir].append(audio_files)

In [3]:
# Read audio signals
f = open('data.csv', 'w')
writer = csv.writer(f)

writer.writerow(['label', 'feature 1', 'feature 2', 'feature 3'])

for dir in files:
    for file in files[dir][0]:
        audio, sr = librosa.core.load(
            path.join(os.getcwd(), '..', 'data', dir, file)
        )

        writer.writerow([dir, audio[0], audio[1], audio[2]])